In [56]:
#from utils2 import getSynteticData, exactTest, MWU, ttests, multiple_plot, getdf
import numpy as np
import pandas as pd
import time
import permutationTest as pt
from scipy.stats import ttest_ind, mannwhitneyu

In [102]:
%load_ext cython
%load_ext rpy2.ipython



The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [103]:
%%R
    runMcCoin <- function(x,y,batches,output,n_samples) {
       e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
            
            DV <- c(x[[i]], y[[i]])
            IV <- factor(rep(c("A", "B"), c(length(x[[i]]), length(y[[i]]))))
            library(coin)                    # for oneway_test(), pvalue()
            output[[i]] <- pvalue(oneway_test(DV ~ IV, distribution=approximate(batches)))
            #output[[i]] <- pvalue(oneway_test(DV ~ IV, distribution="split-up""))
      
            
      
            
            }
    

    return(output)
        
    }

In [114]:
def McCoin(A,B, batches):
    x = pd.DataFrame(A, columns=list(range(A.shape[1]))).T
    y = pd.DataFrame(B, columns=list(range(B.shape[1]))).T
    n_samples = x.shape[1]
    outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T
    
    %R -i x
    %R -i y
    %R -i batches
    %R -i outputDf
    %R -i n_samples
    
    
    %R out <- runMcCoin(x,y, batches, outputDf, n_samples)
    
    
    %R -o out
    return list(out.T["output"].values)



In [115]:
A = [np.array([ 41, 16, 73, 23, 44, 83, 76,  8, 35, 50, 96, 76, 86, 48, 64, 32, 91,
       21, 38, 40, 68,  5, 43, 52, 39, 34, 59, 68, 70, 89, 69, 47, 71, 96,
       84, 32, 67, 81, 53, 77, 51,  5, 91, 64, 80, 50, 40, 47,  9, 51, 16,
        9, 18, 23, 74, 58, 91, 63, 84, 97, 44, 33, 27,  9, 77, 11, 41, 35,
       61, 10, 71, 87, 71, 20, 57, 83,  2, 69, 41, 82, 62, 71, 98, 19, 85,
       91, 88, 23, 44, 53, 75, 73, 91, 92, 97, 17, 56, 22, 44, 94])]

B = [np.array([ 81, 41, 71, 75, 38, 60, 18, 16, 31, 78, 27, 40, 64, 21, 23, 60, 50,
       28,  1, 63, 41, 59, 93, 37, 64, 76, 51, 74,  8, 50, 83,  2, 80, 92,
       97, 50, 41, 47, 60, 74, 79, 95, 96, 33, 17, 22, 44, 59, 99, 22, 50,
       64, 84, 93, 52, 98,  9, 87, 89, 12,  2, 44, 70, 60, 64,  8, 81, 93,
       80, 18, 38, 58, 32, 35, 81, 43, 49, 37, 23, 20, 96, 18, 46, 73, 38,
       10, 22, 37, 40, 84, 91, 37, 41, 20, 57,  9, 79, 50, 52, 28])]

In [116]:
A,B = np.asarray(A), np.asarray(B)

In [117]:
A.shape

(1, 100)

In [118]:
p_val_list5_2_coin = McCoin(A,B,100)

In [137]:
def exactTest(A,B, batches, bins=10, one_side=False):
    return pt.GreenFloatCuda(A,B,bins, batch_size=batches)
    
    
def MWU(A, B, one_side=False):
    p_mw = list()
    for a,b in zip(A, B):
        if one_side:
            p_mw.append(mannwhitneyu(a,b, alternative="less")[1])
        else:
            p_mw.append(mannwhitneyu(a,b, alternative="two-sided")[1])
    return p_mw

def ttests(A,B, one_side=False):
    p_t = list()
    for x, y in zip(A, B):
        t, p = ttest_ind(y, x)
        if one_side:
            p = p/2
            if t<0:
                p = 1-p
        p_t.append(p)
    return p_t

In [161]:
def getPrediction(A,B):
    Data = dict()
    A,B = np.array(A), np.array(B)
    Data["eList"] = exactTest(A,B, int(A.shape[0]/200), 100, False)
    Data["MC_1_000"] = McCoin(A,B, 1_000)
    Data["MC_10_000"] = McCoin(A,B, 10_000)
    Data["MC_100_000"] = McCoin(A,B, 100_000)
    Data["mwuList"] = MWU(A, B, False)
    Data["ttList"] = ttests(A, B, False)
    
    
                                                      
    return Data

    

    

# $A \sim N(0,1)\ and B \sim N(0,1)$. $ |A|=20$ and $|B|=20$ with $2000$ samples.

In [162]:
def getSynteticData(func, setN=20, sampleN=2_000, mean=0, std=1,seed=1):
    """Generate synthetic data"""
    np.random.seed(seed)
    AN, BN = [func(mean,std,setN) for i in range(sampleN)], [func(0,std,setN) for i in range(sampleN)]
    return AN, BN

In [163]:
np.random.seed(10)
S = 2_000_000

In [ ]:
AN, BN = getSynteticData(np.random.normal,mean=0,sampleN=S)
%time DataNormNoDiff = getPrediction(AN, BN)

(2000000, 20)


In [159]:
def getdf(P, num_examples, test=None):
    P.sort()
    p_arr = np.array(P)
    offset = 1.0/float(num_examples)
    ideal_arr = np.linspace(offset,1.0-offset,num_examples)
    if test:
        Pdf = pd.DataFrame({'Observed p-value':p_arr,'Theoretical p-value':ideal_arr, "Test":[test]*ideal_arr.shape[0]})
    return Pdf

In [160]:
normMC_1_000_Df = getdf(DataNormNoDiff["MC_1_000"], S, "Coin Monte Carlo, b=1000")
normMC_10_000_Df = getdf(DataNormNoDiff["MC_10_000"], S, "Coin Monte Carlo, b=10000")
normMC_100_000_Df = getdf(DataNormNoDiff["MC_100_000"], S, "Coin Monte Carlo, b=100000")
normexactDf = getdf(DataNormNoDiff["eList"], S, "Parallel Green")
normttDf = getdf(DataNormNoDiff["ttList"], S, "$\it{T}$ test")
normmwuDf = getdf(DataNormNoDiff["mwuList"], S, "Mann–Whitney $\it{U}$ test")

ValueError: arrays must all be same length

In [ ]:
def multiple_plot(df, save_name):
    sns.set(style="white")
    sns.set_context("talk")
    low = min(df["Theoretical p-value"])/2
    hi = max(df["Theoretical p-value"])
    g=sns.lmplot(x='Theoretical p-value', y ='Observed p-value', data=df, 
                  fit_reg=False, height=7, truncate=True, scatter_kws={"s": 15}, hue="Test")
    
    g.set(xscale="log", yscale="log")
    g._legend.remove()


    plt.xlabel(r'Theoretical $\it{p}$ value', fontsize=24)
    plt.ylabel(r'Observed $\it{p}$ value', fontsize=24)
    
    axes = g.axes
    g.set(ylim=(low,hi), xlim=(low,hi))
    plt.plot([low,hi],[low,hi], "k", linewidth=1)
    plt.plot([2 * low,2 * hi],[low,hi], "--k", linewidth=1)
    plt.plot([low / 2,hi / 2],[low,hi], "--k", linewidth=1)
    sns.set_style("ticks")
    sns.despine()
    g.fig.tight_layout()
    
    
    plt.legend().set_title('')
    plt.legend(loc=2,prop={'size': 24})
    plt.xticks(size = 24)
    plt.yticks(size = 24)


    
    g.savefig(save_name)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
multiple_plot(pd.concat((normMC_1_000_Df, normMC_10_000_Df, normMC_100_000_Df,normexactDf, normttDf, normmwuDf)), "./figures/experiemnt5/allTests_0_1_50")
              
              

# $A \sim logN(0,1)\ and B \sim logN(0,1)$. $ |A|=20$ and $|B|=20$ with $2000$ samples.

In [ ]:
AN, BN = getSynteticData(np.random.lognormal, mean=0, sampleN=S)
DataLogNormNonDiff = getPrediction(AN, BN)

In [ ]:
logfpDf = getdf(DataLogNormNonDiff["fpList"], S, "FastPerm")
logexactDf = getdf(DataLogNormNonDiff["eList"], S, "Parallel Green")
logttDf = getdf(DataLogNormNonDiff["ttList"], S, "$\it{T}$ test")
logmwuDf = getdf(DataLogNormNonDiff["mwuList"], S, "Mann–Whitney $\it{U}$ test")

In [ ]:
multiple_plot(pd.concat((logfpDf, logexactDf, logttDf, logmwuDf)), "figures/calibration/allTestsLog_0_1_50")
